# Chapter 5 - Minimal APIs

### [Minimal API Quick Ref](https://learn.microsoft.com/en-us/aspnet/core/fundamentals/minimal-apis?view=aspnetcore-9.0)

- Inspired by Express and Node.js

## Top-level statements

instead of:

```csharp
using System;

namespace Application
{
    class Program
    {
        static void Main(string[] args)
        {
            Console.WriteLine("Hello World!");
        }
    }
}
```

We can just do this in `Program.cs`

```csharp
Console.WriteLine("Hello World!");
```

## Minimal hosting

```csharp
//Used to configure the application
WebApplicationBuilder builder = Microsoft.AspNetCore.Builder.WebApplication.CreateBuilder(args);
//run the application
WebApplication app = builder.Build();
app.Run();
```

- Lean and easy to get started
- Does not support Razor views like MVC does
- lower resource consumption (Performance boost)
- Great for microservices

Map a route to a delegate (Lambda expression)

```csharp
app.MapGet("some/url/pattern", (HttpContext context) => {
 // Do something
});
```

- Map a default if no routes match using `MapFallback`

### Parameter data binding

- Can be implicit 

```csharp
app.MapGet(
 "minimal-endpoint-input-route-implicit/{id}", 
 (int id) => $"The id was {id}."
);
```

Or explicit

```csharp
app.MapGet(
 "minimal-endpoint-input-route-explicit/{id}",
 ([FromRoute] int id) => $"The id was {id}."
);
```

List of additional options

- [FromRoute]
- [FromQuery]
- [FromHeader]
- [FromBody] - json body
- [FromForm]
- [FromServices]
- Custom [AsParmaters] - Binds form values to a type (names of keys match properties in type)

## Returning results

- Can use `Results` or `TypedResults` both implement `IResult`
    * `TypedResults` is better becuase the OpenAPI docs generated will be better

```csharp
app.MapGet(
 "minimal-endpoint-output-coordinate-ok2/",
 () => TypedResults.Ok(new Coordinate {
 Latitude = 43.653225,
 Longitude = -79.383186
 })
);
```

### TypedResults

- `TypedResults.Problem` returns a 500 Internal Server Error respose but is customizable
- `TypedResults.ValidationProblem` returns a 400 Bad Request respose but allows to have a consitent validation response

> everaging the Problem Details structure improves the interoperability of our API by choosing a standard instead of crafting a custom way of returning our API errors

- `TypedResults.File` write file content to the response stream
- `TypedResults.Bytes` send binary (byte array) to the response
- `Json` - serialize data to json - .Ok also does this but can customize serializer

Multiple results

```csharp
Results<Ok, Conflict> MultipleResultsDelegate()
{
 return Random.Shared.Next(0, 100) % 2 == 0
 ? TypedResults.Ok()
 : TypedResults.Conflict();
}
```

- Methods to add metadata such as [RequireAuthorization]

### Json Serialization

Configure json naming

```csharp
builder.Services.ConfigureHttpJsonOptions(options => {
 options.SerializerOptions.PropertyNamingPolicy = JsonNamingPolicy.
KebabCaseLower;
});
```

### Endpoint filters

- Interceptor for api endpoints
- Could be a great way to add logging / tracing for debugging

```csharp
 .MapGet("good-rating/{rating}", (Rating rating)
 => TypedResults.Ok(new { Rating = rating }))
 .AddEndpointFilter(async (context, next) =>
 {
 var rating = context.GetArgument<Rating>(0);
 if (rating == Rating.Bad)
 {
 return TypedResults.Problem(
 detail: "This endpoint is biased and only accepts positive 
ratings.",
 statusCode: StatusCodes.Status400BadRequest
 );
 }
 return await next(context);
 });
 ```
 ```csharp
 .AddEndpointFilter<GoodRatingFilter>();
 ```

- Filter can be applied to groups

### Enpoint Groups

- Tool to orgainzie endpoints

```csharp
public static class OrganizingEndpoints
{
 public static void MapOrganizingEndpoints(
 this IEndpointRouteBuilder app)
 {
var group = app
 .MapGroup("organizing-endpoints-fluently")
 .WithTags("Organizing Fluent Endpoints")
 ;
 // Map endpoints and groups here
 return app
 }
}
```

```csharp
var builder = WebApplication.CreateBuilder(args);
var app = builder.Build();
app.MapOrganizingEndpoints();
app.Run();
```

**Neat trick**

```csharp
var updatedCustomer = await customerRepository.UpdateAsync(
 customer with { Name = input.Name },
 cancellationToken
 )
```

Seems that the `with` keyword is shorthand for making modified copies, cince records are immutable by default as explained here.

https://programmingpulse.vercel.app/blog/best-practice-start-using-csharp-9-records-for-dtos

```
public record class UpdateCustomer(string Name);
```

